# SPAM or HAM
---

Here I made a model for classifying whether the context given is spam or ham.

The model used here is a trnsformer model from HuggingFace. The model will be used from the 'transformer' library.

Here after loading the model from HuggingFace I did some fine tuning.

---
The Dataset used here is taken from Kaggle and used google colab for writing code.

The dataset originally contains 8000 rows but as it keeps crashing due to RAM limitation in colab I only selected 2000 rows.

---
I tried to use TFTrainer module which can help the model support tensorflow but as for some reason I couldn't import the package, So I ran it as it is.


In [ ]:
#!pip install transformers

###Import the dataset and preprocess the the data

In [ ]:
import tensorflow as tf
import pandas as pd

In [ ]:
df=pd.read_csv('/content/combined_data.csv',nrows=2000)

In [ ]:
df.head()

,label,text
0,1,ounce feather bowl hummingbird opec moment ala...
1,1,wulvob get your medircations online qnb ikud v...
2,0,computer connection from cnn com wednesday es...
3,1,university degree obtain a prosperous future m...
4,0,thanks for all your answers guys i know i shou...


In [ ]:
df.shape

(2000, 2)

In [ ]:
df.isna().sum()

label    0
text     0
dtype: int64

In [ ]:
# make both dependent and independent column a list object
X=list(df['text'])
y=list(df['label'])

In [ ]:
# split the datain to training and testing dataset
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
# load the Tokenizer and model from HuggingFace

from transformers import AutoTokenizer,AutoModelForSequenceClassification

model_path='lxyuan/distilbert-base-multilingual-cased-sentiments-student'
tokenizer=AutoTokenizer.from_pretrained(model_path)
model=AutoModelForSequenceClassification.from_pretrained(model_path)

In [ ]:
model

In [ ]:
model=model.to('cuda')

In [ ]:
X_train_encoded=tokenizer(X_train,padding=True,truncation=True,max_length=256)
X_test_encoded=tokenizer(X_test,padding=True,truncation=True,max_length=256)

In [ ]:
# convert the dataset so it can be fine tuned with the model

import torch

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
train_dataset=Dataset(X_train_encoded,y_train)
test_dataset=Dataset(X_test_encoded,y_test)

### Model creation and training

In [ ]:
from transformers import TrainingArguments,Trainer

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install accelerate -U

In [ ]:
pip install transformers[torch]


In [ ]:
!pip install --upgrade accelerate

In [ ]:
# set args parameter value for Trainer instance
args = TrainingArguments(
    output_dir='./result',
    num_train_epochs=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    gradient_accumulation_steps=8,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10
)

In [ ]:
# Create the tTrainer instance
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    #compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Step,Training Loss
10,1.584800
20,1.447300
30,1.253600
40,1.099400
50,0.912300
60,0.766800
70,0.728500
80,0.661300
90,0.636800
100,0.558700


TrainOutput(global_step=100, training_loss=0.9649577569961548, metrics={'train_runtime': 80.2094, 'train_samples_per_second': 39.896, 'train_steps_per_second': 1.247, 'total_flos': 211951617638400.0, 'train_loss': 0.9649577569961548, 'epoch': 2.0})

### Evaluation and prediction

In [ ]:
trainer.evaluate()

{'eval_loss': 0.5175684094429016,
 'eval_runtime': 3.1794,
 'eval_samples_per_second': 125.809,
 'eval_steps_per_second': 15.726,
 'epoch': 2.0}

In [ ]:
# predict the testing data
prediction=trainer.predict(test_dataset)[1]

In [ ]:
prediction

array([1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0,

In [ ]:
from sklearn.metrics import confusion_matrix

cf_mat=confusion_matrix(y_test,prediction)

In [ ]:
cf_mat

array([[175,   0],
       [  0, 225]])

###Saving the model

In [ ]:
trainer.save_model('classifier')

In [ ]:
model=trainer.model
model.save_pretrained("my_sentiment_model")


In [ ]:
# for loading the model
'''
from transformers import TFAutoModelForSequenceClassification

# Load the saved model
loaded_model = TFAutoModelForSequenceClassification.from_pretrained("my_sentiment_model")
'''

'\nfrom transformers import TFAutoModelForSequenceClassification\n\n# Load the saved model\nloaded_model = TFAutoModelForSequenceClassification.from_pretrained("my_sentiment_model")\n'